"Three electric power plants with capacities of 25, 40, and 30 million kWh supply electricity
to three cities. The maximum demands at the three cities are estimated at 30, 35, and
25 million kWh. The price per million kWh at the three cities is given in Table 5.25.
During the month of August, there is a 20% increase in demand at each of the three cities,
which can be met by purchasing electricity from another network at a premium rate
of $1000 per million kWh. The network is not linked to city 3, however. The utility company
wishes to determine the most economical plan for the distribution and purchase of
additional energy.
(a) Formulate the problem as a transportation model.
(b) Determine an optimal distribution plan for the utility company.
(c) Determine the cost of the additional power purchased by each of the three cities."

Operations Research An Introduction by Hamdy A. Taha (z-lib.org)



In [1]:
from pulp import *  # Import pulp libray
import pandas as pd

In [2]:
# Define classes for the sinks and sources

# For each city added in the model, choose the maximum demands 

class City:
    def __init__(self, max_demand, demand_increase):
        self.max_demand = max_demand
        self.demand_increase = demand_increase
        
    def surplus_demand (self):
        surplus_demand = self.max_demand * ( self.demand_increase/100 )
        return surplus_demand
            


# For each power plant added in the model, choose its capacities to supply

class Source:
    
    def __init__(self, capacity):
        self.capacity = capacity

class Network:
    
    def __init__(self, loss):
        self.loss = loss
        


In [10]:
# Choose the cities' attributes
city1 = City(30, 20).max_demand
city2 = City(35, 20).max_demand
city3 = City(25, 20).max_demand

#Calculate the surplus demand

surplus1 = City(30, 20).surplus_demand()
surplus2 = City(35, 20).surplus_demand()
surplus3 = City(25, 20).surplus_demand()



# Choose the power plants' attributes
power_plant1 = Source(25)
power_plant2 = Source(40)
power_plant3 = Source(30)

## Choose the Networks' loss

network11 = Network (0)
network12 = Network (0)
network13 = Network (0)
network21 = Network (0)
network22 = Network (0)
network23 = Network (0)
network31 = Network (0)
network32 = Network (0)
network33 = Network (0)


# Calculate transportation costs per million kWh supply electricity

# Define the generic expression that calculates cost prices from each power plant to each city

one_one = 600
one_two = 700
one_three = 400
two_one = 320
two_two = 300
two_three = 350
three_one = 500
three_two = 480
three_three = 450


30

In [22]:
# Add the variables for the model - xij is the arc the connects the supplier i to city j

prob = LpProblem("Min", LpMinimize)

x11 = LpVariable("million kWh 1-1", 0, cat='Continuous')
x12 = LpVariable("million kWh 1-2", 0, cat='Continuous')
x13 = LpVariable("million kWh 1-3", 0, cat='Continuous')

x21 = LpVariable("million kWh 2-1", 0, cat='Continuous')
x22 = LpVariable("million kWh 2-2", 0, cat='Continuous')
x23 = LpVariable("million kWh 2-3", 0, cat='Continuous')

x31 = LpVariable("million kWh 3-1", 0, cat='Continuous')
x32 = LpVariable("million kWh 3-2", 0, cat='Continuous')
x33 = LpVariable("million kWh 3-3", 0, cat='Continuous')

outside_net1 = LpVariable("million kWh outside network - 1", 0, cat='Continuous')
outside_net2 = LpVariable("million kWh outside network - 2", 0, cat='Continuous')


#the increase in demand at each of the three cities, can be met by purchasing electricity from another network at a premium rate of $1000 per million kWh
cost = 1000

# Add the objective function to the problem

prob += one_one * x11 * (1 - network11.loss) + one_two * x12 * (1 - network12.loss) + one_three * x13 * (1 - network13.loss) + \
+ two_one * x21 * (1 - network21.loss) + two_two * x22 * (1 - network22.loss) + two_three * x23 * (1 - network23.loss) + \
three_one * x31 * (1 - network31.loss) + three_two * x32 * (1 - network32.loss) + three_three * x33 * (1 - network33.loss) +\
outside_net1 * cost + outside_net2 * cost , "objective function"

#  Add constraints to the problem : the total number of energy supply from the provider to each demanding agency (xij) must be less or equal to the sourcer's capacity to supply (ci)

prob += x11 + x12 + x13 <= power_plant1.capacity, "power_plant_c1"
prob += x21 + x22 + x23 <= power_plant2.capacity, "power_plant_c2"
prob += x31 + x32 + x33 <= power_plant3.capacity, "power_plant_c3"


# Add constraints to the problem: the total number of cars supplied from the provider to each demanding agency (xij) must be equal to the sink's necessity (cj)

prob += x11 + x21 + x31 + outside_net1 == (city1 + surplus1), "city_c1"
prob += x12 + x22 + x23 + outside_net2 == (city2 + surplus2), "city_c2"
prob += x13 + x23 + x33 == (city3 + surplus3), "city_c3"

prob += x11 + x12 + x13 + x21 + x22 + x23 + x31 + x32 + x33 + outside_net1 + outside_net2 == city1 + surplus1 + city2 + \
surplus2 + city3 + surplus3




In [23]:
# Solution
prob.solve()

print("Status:", LpStatus[prob.status])  # solution status



obj = value(prob.objective)
print("The total cost of transportation is: ${}".format(round(obj, 2)))


prob.writeLP("Energy_Problem.lp")
prob.solve(GLPK(options=['--ranges sensit1.sen']))



for v in prob.variables():
    print(v.name, "=", v.varValue, "\tReduced Costs = ", v.dj)
    

    
s=[{'name': name, 'Shadow Price': c.pi, "Slack": c.slack} for name, c in prob.constraints.items()] ## sensitivity analysis
print(pd.DataFrame(s))


Status: Optimal
The total cost of transportation is: $49750.0
million_kWh_1_1 = 0.0 	Reduced Costs =  150.0
million_kWh_1_2 = 0.0 	Reduced Costs =  250.0
million_kWh_1_3 = 25.0 	Reduced Costs =  0.0
million_kWh_2_1 = 0.0 	Reduced Costs =  20.0
million_kWh_2_2 = 40.0 	Reduced Costs =  0.0
million_kWh_2_3 = 0.0 	Reduced Costs =  50.0
million_kWh_3_1 = 25.0 	Reduced Costs =  0.0
million_kWh_3_2 = 0.0 	Reduced Costs =  30.0
million_kWh_3_3 = 5.0 	Reduced Costs =  0.0
million_kWh_outside_network___1 = 11.0 	Reduced Costs =  0.0
million_kWh_outside_network___2 = 2.0 	Reduced Costs =  0.0
             name  Shadow Price  Slack
0  power_plant_c1        -550.0   -0.0
1  power_plant_c2        -700.0   -0.0
2  power_plant_c3        -500.0   -0.0
3         city_c1          50.0   -0.0
4         city_c2          50.0   -0.0
5         city_c3           0.0   -0.0
6             _C1         950.0   -0.0
